# Fitsio-Subset

In [37]:
import numpy as np
import time
from astropy.io import fits

def printname(name):
    print name

## how many spectra to read? 
nspectra = 1000
totnspectra = 1000

### first, read the pmf file
pmf = np.zeros( (nspectra,3))
i = -1
for line in open("data/pmf1k"):
    if "plates" in line:
        continue
    cols = line.split()

    i+=1
    if i>nspectra:
        continue

    pmf[i][0] = int(cols[0])
    pmf[i][1] = int(cols[1])
    pmf[i][2] = int(cols[2])

print pmf.shape

## this goes plate/mjd/fiber. I want to pull out spectrum - so plate/mjd/fiber/coadd/...
spectra = []

### timeing: open file, extract spectra, put into the array
t1 = time.clock()
for i in range(nspectra):

    #datasetname = "/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/v5_7_0/"
    #+str(pmf[i][0])[:-2]+"/spPlate-"+str(pmf[i][0])[:-2]+"-"+str(pmf[i][1])[:-2]+".fits"
    datasetname = "fits-1k/spPlate-"+str(pmf[i][0])[:-2]+"-"+str(pmf[i][1])[:-2]+".fits"
    try:
        hdu = fits.open(datasetname)

    except:
        print "******* dataset not found! **************", datasetname
        continue
    
    flux = hdu[0].data
    
    try:
        spectra.append(flux[pmf[i][2]])
    except:
        print "failed to append spectra"
        print i, pmf[i][2], flux.shape

t2 = time.clock()
spectra = np.array(spectra)
print spectra.shape
print spectra[0]
print "time taken to load all the spectra into an array:", t2-t1, "sec"

IOError: [Errno 2] No such file or directory: 'data/pmf1k'

# parse PMF

In [38]:
def parsePMF(pmflist):
    try:
        df = pd.read_csv(pmflist,delimiter=' ',names=["plates","mjds","fibers"],index_col=None,dtype=str)
        df = df.sort(['plates'],ascending=[1])
        plates = list(map(str,df['plates'].values))
        mjds = list(map(str,df['mjds'].values))
        fibers = list(map(str,df['fibers'].values))
    except Exception as e:
        print("pmf csv read error or not exist:%s"%e,pmflist)
        print("e.g., 1st row of csv should be 'plates mjds fibers'")
    return (plates,mjds,fibers)

# Using astropy.io fits

In [3]:
import numpy as np
import time
from astropy.io import fits
import traceback
boss_dir="/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/v5_7_0/"
i_wave={"flux": 1,"ivar": 0,"andmask": 2, "ormask": 3, "wavedisp": 4, "sky": 6}
def fitsread_pmf(plate,mjd,fiber,wave):
    '''
        para: plate, mjd, fiber
        para: wave is the name of wavelength variable
        return: fiber-th row in dataset wave
    '''
    try:
        fitsfile=boss_dir+str(plate)+"/spPlate-"+str(plate)+"-"+str(mjd)+".fits"
        hdu = fits.open(fitsfile)
    except Exception as e:
        traceback.print_exc()
        print("fits open error %s"%fitsfile)
    try:
        di=int(i_wave[wave])
        dwave=hdu[di].data[int(fiber)]
    except Exception as e:
        traceback.print_exc()
        print("extraction error")
    return dwave


# Using fitsio

In [33]:
import numpy as np
import sys
sys.path.append('/global/homes/j/jialin/.local/edison/2.7-anaconda/lib/python2.7/site-packages')
import time
import fitsio
import traceback
boss_dir="/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/v5_7_0/"
i_wave={"flux": 1,"ivar": 0,"andmask": 2, "ormask": 3, "wavedisp": 4, "sky": 6}
def fitsread_pmf(plate,mjd,fiber,wave):
    '''
        para: plate, mjd, fiber
        para: wave is the name of wavelength variable
        return: fiber-th row in dataset wave
    '''
    try:
        fitsfile=boss_dir+str(plate)+"/spPlate-"+str(plate)+"-"+str(mjd)+".fits"
        dfits=fitsio.FITS(fitsfile)
    except Exception as e:
        traceback.print_exc()
        print("fits open error %s"%fitsfile)
    try:
        di=int(i_wave[wave])
        print ("%dth variable %s, %dth fiber"%(di,wave,fiber))
        dwave=dfits[di][int(fiber)-1:int(fiber),:]
    except Exception as e:
        traceback.print_exc()
        print("extraction error")
    return dwave

# test

In [34]:
'''
3973 55323 790
5329 55946 618
4860 55691 325
4179 55684 141
6474 56362 402
6170 56240 672
6832 56426 160
4395 55828 411
7340 56726 310
7137 56566 527
'''

'\n3973 55323 790\n5329 55946 618\n4860 55691 325\n4179 55684 141\n6474 56362 402\n6170 56240 672\n6832 56426 160\n4395 55828 411\n7340 56726 310\n7137 56566 527\n'

In [35]:
plate=3973
mjd=55323
fiber=790
dw=fitsread_pmf(plate,mjd,fiber,'flux')
dw

1th variable flux, 790th fiber


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [36]:
plate=3973
mjd=55323
fiber=5
dw=fitsread_pmf(plate,mjd,fiber,'flux')
dw

1th variable flux, 5th fiber


array([[ 0.      ,  0.067215,  0.      , ...,  0.      ,  0.      ,  0.      ]], dtype=float32)

In [39]:
def test_fitsread_pmf(pmflist,var):
    try:
        plate,mjd,fiber=parsePMF(pmflist)
    except Exception as e:
        traceback.print_exc()
        print("parse pmf: %s error"%pmflist)
    print ("reading %d pmf"%length(plate))
    try:
        for i in range(0,length(plate)):
            fitsread_pmf(plate[i],mjd[i],fiber[i],var)
    except Exception as e:
        traceback.print_exc()
        print("read fiber %d error %s"%(fiber[i],var))

boss2hdf5_v2.ipynb   H5bossIO-subset.ipynb
Fitsio-Subset.ipynb  h5boss-test.ipynb
